# Natural Images Classification using CNN
## CECS 456 - Deep Learning Project

**Dataset:** Natural Images (8 Classes)
- airplane, car, cat, dog, flower, fruit, person, motorbike

**Objective:** Build and train a CNN to classify natural images into 8 categories

## 1. Setup and Import Libraries

In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import os

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

## 2. Upload Dataset

**Instructions:**
1. Download the Natural Images dataset from: https://www.kaggle.com/datasets/prasunroy/natural-images
2. Upload the zip file using the cell below
3. Wait for extraction to complete

In [ ]:
# Upload dataset zip file
from google.colab import files
import zipfile

print("Please upload your natural-images.zip file:")
uploaded = files.upload()

# Extract the zip file
for filename in uploaded.keys():
    print(f"Extracting {filename}...")
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall('/content/')
    print("Extraction complete!")

# Set the path to dataset (adjust if needed)
data_dir = '/content/natural_images'

# Verify the structure
if os.path.exists(data_dir):
    print(f"\nDataset found at: {data_dir}")
    print(f"Classes: {os.listdir(data_dir)}")
else:
    print("ERROR: Dataset not found. Please check the path.")

## 3. Explore the Dataset

In [ ]:
# Count images in each class
classes = sorted(os.listdir(data_dir))
print(f"Number of classes: {len(classes)}")
print(f"Class names: {classes}\n")

total_images = 0
print("Images per class:")
print("-" * 30)
for class_name in classes:
    class_path = os.path.join(data_dir, class_name)
    if os.path.isdir(class_path):
        num_images = len([f for f in os.listdir(class_path) if f.endswith(('.jpg', '.jpeg', '.png'))])
        total_images += num_images
        print(f"{class_name:15s}: {num_images:4d} images")

print("-" * 30)
print(f"Total images: {total_images}")

## 4. Visualize Sample Images

In [ ]:
# Display sample images from each class
plt.figure(figsize=(15, 10))

for i, class_name in enumerate(classes):
    class_path = os.path.join(data_dir, class_name)
    image_files = [f for f in os.listdir(class_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
    
    if image_files:
        # Load first image from this class
        img_path = os.path.join(class_path, image_files[0])
        img = plt.imread(img_path)
        
        plt.subplot(2, 4, i + 1)
        plt.imshow(img)
        plt.title(f"{class_name}\n({len(image_files)} images)", fontsize=12)
        plt.axis('off')

plt.tight_layout()
plt.savefig('sample_images.png', dpi=150, bbox_inches='tight')
plt.show()

print("Sample images saved as 'sample_images.png'")

## 5. Prepare Data with Augmentation

Data augmentation helps prevent overfitting by creating variations of training images.

In [ ]:
# Set parameters
IMG_SIZE = 150
BATCH_SIZE = 32

# Create data generators with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,              # Normalize pixel values to [0,1]
    validation_split=0.2,        # Use 20% for validation
    rotation_range=20,           # Rotate images randomly up to 20 degrees
    width_shift_range=0.2,       # Shift horizontally by 20%
    height_shift_range=0.2,      # Shift vertically by 20%
    horizontal_flip=True,        # Flip images horizontally
    zoom_range=0.2,              # Zoom in/out by 20%
    fill_mode='nearest'          # Fill in missing pixels after transformations
)

# Training set (with augmentation)
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

# Validation set (no augmentation, only rescaling)
val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

val_generator = val_datagen.flow_from_directory(
    data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=42
)

print(f"\nTraining samples: {train_generator.samples}")
print(f"Validation samples: {val_generator.samples}")
print(f"Number of classes: {train_generator.num_classes}")
print(f"Class indices: {train_generator.class_indices}")

## 6. Build CNN Model

Architecture:
- 4 Convolutional blocks (Conv2D + MaxPooling)
- Progressively increasing filters: 32 → 64 → 128 → 128
- Dropout for regularization
- Dense layers for classification

In [ ]:
# Build the CNN model
model = keras.Sequential([
    # Input layer
    keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
    
    # First convolutional block
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    
    # Second convolutional block
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    
    # Third convolutional block
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    
    # Fourth convolutional block
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    
    # Flatten and dense layers
    layers.Flatten(),
    layers.Dropout(0.5),                              # Prevent overfitting
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')  # Output layer
], name='Natural_Images_CNN')

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Display model architecture
model.summary()

# Count total parameters
total_params = model.count_params()
print(f"\nTotal trainable parameters: {total_params:,}")

## 7. Train the Model

This will take approximately 15-20 minutes on GPU.

In [ ]:
# Set training parameters
EPOCHS = 25

print(f"Starting training for {EPOCHS} epochs...\n")

# Train the model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    verbose=1
)

print("\nTraining complete!")

# Save the trained model
model.save('natural_images_cnn_model.h5')
print("Model saved as 'natural_images_cnn_model.h5'")

## 8. Visualize Training Results

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot accuracy
axes[0].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
axes[0].set_title('Model Accuracy Over Epochs', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Plot loss
axes[1].plot(history.history['loss'], label='Training Loss', linewidth=2)
axes[1].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[1].set_title('Model Loss Over Epochs', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Loss', fontsize=12)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_results.png', dpi=150, bbox_inches='tight')
plt.show()

print("Training plots saved as 'training_results.png'")

# Print final metrics
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]

print(f"\n{'='*50}")
print("FINAL RESULTS")
print(f"{'='*50}")
print(f"Training Accuracy:   {final_train_acc:.4f} ({final_train_acc*100:.2f}%)")
print(f"Validation Accuracy: {final_val_acc:.4f} ({final_val_acc*100:.2f}%)")
print(f"Training Loss:       {final_train_loss:.4f}")
print(f"Validation Loss:     {final_val_loss:.4f}")
print(f"{'='*50}")

## 9. Detailed Evaluation & Confusion Matrix

In [ ]:
# Get predictions on validation set
print("Generating predictions on validation set...")
val_generator.reset()
predictions = model.predict(val_generator, steps=len(val_generator), verbose=1)
predicted_classes = np.argmax(predictions, axis=1)

# Get true labels
true_classes = val_generator.classes
class_labels = list(val_generator.class_indices.keys())

# Print classification report
print("\n" + "="*70)
print("CLASSIFICATION REPORT")
print("="*70)
print(classification_report(true_classes, predicted_classes, target_names=class_labels, digits=4))

# Calculate overall accuracy
correct_predictions = np.sum(predicted_classes == true_classes)
total_predictions = len(true_classes)
overall_accuracy = correct_predictions / total_predictions

print(f"\nOverall Validation Accuracy: {overall_accuracy:.4f} ({overall_accuracy*100:.2f}%)")
print(f"Correct Predictions: {correct_predictions}/{total_predictions}")

In [ ]:
# Create confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Plot confusion matrix
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_labels, yticklabels=class_labels,
            cbar_kws={'label': 'Number of Predictions'})
plt.title('Confusion Matrix - Natural Images Classification', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

print("Confusion matrix saved as 'confusion_matrix.png'")

# Calculate per-class accuracy
print("\nPer-Class Accuracy:")
print("-" * 40)
for i, class_name in enumerate(class_labels):
    class_correct = cm[i, i]
    class_total = cm[i, :].sum()
    class_acc = class_correct / class_total if class_total > 0 else 0
    print(f"{class_name:15s}: {class_acc:.4f} ({class_acc*100:.2f}%)")

## 10. Test Predictions on Sample Images

In [ ]:
# Get some sample predictions
val_generator.reset()
sample_batch = next(val_generator)
sample_images = sample_batch[0][:9]  # First 9 images
sample_labels = sample_batch[1][:9]

# Make predictions
sample_predictions = model.predict(sample_images)
sample_pred_classes = np.argmax(sample_predictions, axis=1)
sample_true_classes = np.argmax(sample_labels, axis=1)

# Plot predictions
fig, axes = plt.subplots(3, 3, figsize=(12, 12))
axes = axes.ravel()

for i in range(9):
    axes[i].imshow(sample_images[i])
    
    true_label = class_labels[sample_true_classes[i]]
    pred_label = class_labels[sample_pred_classes[i]]
    confidence = sample_predictions[i][sample_pred_classes[i]] * 100
    
    # Color code: green if correct, red if wrong
    color = 'green' if sample_true_classes[i] == sample_pred_classes[i] else 'red'
    
    axes[i].set_title(
        f"True: {true_label}\nPred: {pred_label}\nConf: {confidence:.1f}%",
        color=color, fontsize=10, fontweight='bold'
    )
    axes[i].axis('off')

plt.tight_layout()
plt.savefig('sample_predictions.png', dpi=150, bbox_inches='tight')
plt.show()

print("Sample predictions saved as 'sample_predictions.png'")

## 11. Download Results

Download all generated files for your report and GitHub.

In [ ]:
# List of files to download
files_to_download = [
    'natural_images_cnn_model.h5',
    'sample_images.png',
    'training_results.png',
    'confusion_matrix.png',
    'sample_predictions.png'
]

print("Downloading files...\n")
for filename in files_to_download:
    if os.path.exists(filename):
        files.download(filename)
        print(f"✓ Downloaded: {filename}")
    else:
        print(f"✗ Not found: {filename}")

print("\nAll files downloaded!")
print("\nNext steps:")
print("1. Upload these files to your GitHub repository")
print("2. Use the results in your project report")
print("3. Make sure to cite the accuracy values shown above")

## 12. Project Summary

**What We Did:**
1. Loaded and explored the Natural Images dataset (8 classes)
2. Applied data augmentation to prevent overfitting
3. Built a CNN with 4 convolutional blocks
4. Trained for 25 epochs with validation
5. Evaluated performance with confusion matrix
6. Analyzed per-class accuracy

**Key Findings:**
- Model achieved XX% validation accuracy
- Best performing classes: [based on confusion matrix]
- Most confused classes: [based on confusion matrix]
- Data augmentation helped generalization

**Files Generated:**
- `natural_images_cnn_model.h5` - Trained model
- `sample_images.png` - Dataset samples
- `training_results.png` - Training curves
- `confusion_matrix.png` - Classification performance
- `sample_predictions.png` - Example predictions

**Remember to:**
- Update your report with the actual accuracy values
- Analyze which classes were confused and why
- Discuss potential improvements
- Upload everything to GitHub

---
**Author:** [Your Name]

**Course:** CECS 456 - Machine Learning

**Date:** [Current Date]